<a href="https://www.kaggle.com/code/tornikeonoprishvili/download-youtube-channel?scriptVersionId=99030467" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Dowload entire youtube channel like a pro(grammer)

This notebook will download all the contents from a specified youtube channel URL. Sorted in the most-to-least viewed fashion, with a limit of N downloads. 

In [ ]:
CHANNEL_URL = 'https://www.youtube.com/c/Monstercat/videos?view=0&sort=p&flow=grid'
FORMAT = 'mp3'
NUMBER_OF_VIDEOS = 50
SORT_BY_VIEWS = True
CORES = 16

In [ ]:
from bs4 import BeautifulSoup
import os, time
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from IPython.display import FileLink
import shutil

Install all required tools

In [ ]:
!sudo apt-get update
!sudo apt-get install -y unzip xvfb libxi6 libgconf-2-4
!sudo apt-get -y install firefox

!sudo curl -L https://yt-dl.org/downloads/latest/youtube-dl -o /usr/local/bin/youtube-dl
!sudo chmod a+rx /usr/local/bin/youtube-dl

!pip install webdriver-manager
!pip install selenium

!wget https://github.com/mozilla/geckodriver/releases/download/v0.31.0/geckodriver-v0.31.0-linux64.tar.gz
!tar -xvzf geckodriver-v0.31.0-linux64.tar.gz    
!chmod +x geckodriver
os.environ['PATH'] += ':'+os.getcwd()

In [ ]:
options = Options()
options.headless = True
driver = webdriver.Firefox(options=options)
driver.get(CHANNEL_URL)

NUM_RETRIES = 10
anchor_list = []
while len(anchor_list) < NUMBER_OF_VIDEOS:
    driver.execute_script("window.scrollTo(0, 100000000);")
    retries = 0
    while True:
        time.sleep(0.3)
        page_source = driver.page_source
        soup = BeautifulSoup(page_source)
        current_list = soup.find_all(id='video-title')
        if len(current_list) > len(anchor_list): 
            anchor_list = current_list
            break # Go down 
        retries += 1
    if retries >= NUM_RETRIES:
        break
    print(f'GATHERING LINKS: {len(anchor_list)}/{NUMBER_OF_VIDEOS}')
driver.close()

In [ ]:
base = 'https://www.youtube.com'
vid_links = [base + a['href'] for a in anchor_list]

!mkdir -p Music
for vid_link in tqdm(vid_links):
    !cd Music && youtube-dl --download-archive done.txt --ignore-errors --extract-audio \
               --audio-format {FORMAT} --restrict-filenames --output "%(title)s.%(ext)s" {vid_link}

shutil.make_archive('Music', 'zip', 'Music')
print("Here's all your music, zipped!")
FileLink(r'Music.zip')

In [ ]:

# from multiprocessing import Process

# def run_cpu_tasks_in_parallel(tasks):
#     running_tasks = [Process(target=task) for task in tasks]
#     for running_task in running_tasks:
#         running_task.start()
#     for running_task in running_tasks:
#         running_task.join()

In [ ]:
# @dask.delayed
# def download_one(vid_link):
#     print(f'Downloading {vid_link}')
#     try:
#         !youtube-dl --download-archive done.txt --ignore-errors --extract-audio \
#            --audio-format mp3 --restrict-filenames --output "%(title)s.%(ext)s" {vid_link}
#     except Exception as ex:
#         print(ex)
#     return vid_link

In [ ]:
# !pip install youtube_dl

In [ ]:
# from multiprocessing import Process

# def runInParallel(*fns):
#   proc = []
#   for fn in fns:
#     p = Process(target=fn)
#     p.start()
#     proc.append(p)
#   for p in proc:
#     p.join()

# runInParallel(lambda: print('a'), lambda: print('b'))

In [ ]:
# import youtube_dl

# try:
#     %pushd Music
#     def make_fn(vid_links):
#         def runnable():
#             ytdl_opts= {
#                         'format': 'bestaudio/best',
#                         'postprocessors': [{
#                             'key': 'FFmpegExtractAudio',
#                             'preferredcodec': 'mp3',
#                             'preferredquality': '192',
#                         }],
#                       }
#             ydl_opts = {}
#             with youtube_dl.YoutubeDL(ydl_opts) as ydl:
#                 ydl.download(vid_links)
#         return runnable
#     from itertools import islice
#     def chunk(it, size):
#         it = iter(it)
#         return iter(lambda: tuple(islice(it, size)), ())

#     link_chunks = list(chunk(vid_links, CORES))
#     runnables = [make_fn(chunk) for chunk in link_chunks]
#     runInParallel(*runnables)
# finally:
#     %popd